In [1]:
from z3 import *

def solve_evrp():
    # Problem Parameters
    num_vehicles = 3
    num_nodes = 5
    depot = 0
    
    # Create Z3 Solver
    solver = Optimize()
    
    # Decision Variables
    # x[v][i][j] = 1 if vehicle v travels from node i to node j
    x = [[[Bool(f'x_{v}_{i}_{j}') for j in range(num_nodes)] 
          for i in range(num_nodes)] 
          for v in range(num_vehicles)]
    
    # Energy levels for each vehicle at each node
    energy = [[Real(f'energy_{v}_{i}') for i in range(num_nodes)] 
              for v in range(num_vehicles)]
    
    # Load carried by each vehicle
    load = [[Real(f'load_{v}_{i}') for i in range(num_nodes)] 
            for v in range(num_vehicles)]
    
    # Constraints
    
    # 1. Flow Conservation
    for v in range(num_vehicles):
        for i in range(num_nodes):
            # Number of incoming edges = outgoing edges
            incoming = Sum([x[v][j][i] for j in range(num_nodes) if j != i])
            outgoing = Sum([x[v][i][j] for j in range(num_nodes) if j != i])
            solver.add(incoming == outgoing)
    
    # 2. Each vehicle starts and ends at depot
    for v in range(num_vehicles):
        solver.add(Sum([x[v][depot][j] for j in range(num_nodes) if j != depot]) == 1)
        solver.add(Sum([x[v][j][depot] for j in range(num_nodes) if j != depot]) == 1)
    
    # 3. Vehicle Capacity Constraint
    max_capacity = 1000
    for v in range(num_vehicles):
        for i in range(num_nodes):
            solver.add(load[v][i] >= 0)
            solver.add(load[v][i] <= max_capacity)
    
    # 4. Energy Constraints
    max_energy = 500
    min_energy_threshold = 50
    energy_consumption_rate = 0.05
    
    for v in range(num_vehicles):
        # Initial energy
        solver.add(energy[v][depot] == max_energy)
        
        # Energy consumption constraint
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j:
                    # Energy reduction based on distance and load
                    distance = 10  # Example distance
                    energy_consumed = energy_consumption_rate * distance * load[v][i]
                    solver.add(energy[v][j] == energy[v][i] - energy_consumed)
                    
                    # Ensure energy doesn't fall below threshold
                    solver.add(energy[v][j] >= min_energy_threshold)
    
    # Objective Function
    # Minimize total distance traveled
    total_distance = Sum([
        If(x[v][i][j], 10, 0)  # Example distance of 10 between nodes
        for v in range(num_vehicles)
        for i in range(num_nodes)
        for j in range(num_nodes)
    ])
    
    solver.minimize(total_distance)
    
    # Check and print solution
    if solver.check() == sat:
        model = solver.model()
        print("Solution Found:")
        # Extract and print solution details
        return model
    else:
        print("No solution exists")
        return None

# Run the solver
solve_evrp()

Solution Found:


[x_2_2_4 = False,
 x_2_1_2 = False,
 x_1_2_1 = False,
 x_2_2_1 = False,
 x_2_2_3 = False,
 x_2_4_3 = False,
 x_2_4_1 = False,
 x_2_4_4 = False,
 x_2_4_2 = False,
 x_2_3_4 = False,
 x_0_4_2 = False,
 x_1_2_2 = False,
 x_0_0_1 = True,
 x_0_0_2 = False,
 x_0_0_3 = False,
 x_0_0_4 = False,
 x_0_1_0 = True,
 x_0_2_0 = False,
 x_0_3_0 = False,
 x_0_4_0 = False,
 x_1_0_1 = True,
 x_1_0_2 = False,
 x_1_0_3 = False,
 x_1_0_4 = False,
 x_1_1_0 = True,
 x_1_2_0 = False,
 x_1_3_0 = False,
 x_1_4_0 = False,
 x_2_0_1 = False,
 x_2_0_2 = False,
 x_2_0_3 = False,
 x_2_0_4 = True,
 x_2_1_0 = False,
 x_2_2_0 = False,
 x_2_3_0 = False,
 x_2_4_0 = True,
 x_2_1_1 = False,
 x_2_2_2 = False,
 x_1_3_3 = False,
 x_2_0_0 = False,
 x_2_3_3 = False,
 x_1_4_4 = False,
 x_0_3_2 = False,
 x_0_2_3 = False,
 x_1_4_2 = False,
 x_1_3_2 = False,
 x_0_2_4 = False,
 x_1_1_1 = False,
 x_1_0_0 = False,
 x_0_4_4 = False,
 x_0_3_3 = False,
 x_0_2_2 = False,
 x_0_1_1 = False,
 x_0_0_0 = False,
 x_0_1_2 = False,
 x_0_1_3 = False,
 x_1_3_1 = False,
 x_1_4_1 = False,
 x_1_1_4 = False,
 x_1_2_4 = False,
 x_0_2_1 = False,
 x_0_3_1 = False,
 x_0_4_1 = False,
 x_0_1_4 = False,
 x_1_2_3 = False,
 x_1_4_3 = False,
 x_1_3_4 = False,
 x_2_1_3 = False,
 x_2_3_2 = False,
 x_0_4_3 = False,
 x_0_3_4 = False,
 x_1_1_3 = False,
 x_1_1_2 = False,
 x_2_3_1 = False,
 x_2_1_4 = False,
 energy_1_3 = 500,
 energy_1_0 = 500,
 energy_2_1 = 500,
 energy_1_1 = 500,
 load_0_2 = 0,
 load_0_0 = 0,
 load_2_4 = 0,
 energy_1_2 = 500,
 load_1_4 = 0,
 energy_0_3 = 500,
 load_0_3 = 0,
 load_2_0 = 0,
 load_2_2 = 0,
 energy_0_1 = 500,
 load_2_1 = 0,
 load_1_1 = 0,
 energy_1_4 = 500,
 load_2_3 = 0,
 load_1_3 = 0,
 energy_0_2 = 500,
 load_1_2 = 0,
 energy_2_0 = 500,
 load_0_1 = 0,
 load_0_4 = 0,
 energy_0_4 = 500,
 energy_2_2 = 500,
 load_1_0 = 0,
 energy_2_3 = 500,
 energy_2_4 = 500,
 energy_0_0 = 500]